In [ ]:
from terratorch.datasets.od_aed_elephant import ElephantCocoDataset

In [ ]:
#ds = ElephantCocoDataset(
#    img_folder='/home/romeokienzler/Downloads/AED/training_images',
#    ann_file='/home/romeokienzler/Downloads/AED/annotations_elephants_training.json',
#)
img_folder='/dccstor/terratorch/shared/datasets/aed-elephant/AED/training_images'
ann_file='/dccstor/terratorch/shared/datasets/aed-elephant/annotations_elephants_training.json'

ds = ElephantCocoDataset(
    img_folder=img_folder,
    ann_file=ann_file,
)


In [ ]:
len(ds)

In [ ]:
e = ds[0]

In [ ]:
e

In [ ]:
h, w = e['image'].shape[-2:]

In [ ]:
w

In [ ]:
from terratorch.datamodules.od_aed_elephant import ElephantDataModule
dm = ElephantDataModule(img_folder=img_folder, ann_file=ann_file, min_size=(3648, 5472), tile_size=(512,512), overlap=128, batch_size=8, num_workers=8, save_png=True)


  

In [ ]:
dm.setup()

In [ ]:
tdl = dm.train_dataloader()

In [ ]:
len(tdl.dataset)

In [ ]:
tdl.dataset[0].keys()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms.functional as F
import torch

def show_from_datamodule(dm, n=3, split="train"):
    """Show exactly n samples with bounding boxes from a DataModule."""
    # pick the right loader
    if split == "train":
        loader = dm.train_dataloader()
    elif split == "val":
        loader = dm.val_dataloader()
    elif split == "test":
        loader = dm.test_dataloader()
    else:
        raise ValueError("split must be 'train', 'val', or 'test'")
    
    shown = 0
    for batch in loader:  # loop until enough samples found
        images = batch["image"]     # (B, C, H, W)
        boxes  = batch["boxes"]     # list[Tensor] or padded tensor

        for img, b in zip(images, boxes):
            # skip samples with no boxes
            if b is None or (isinstance(b, torch.Tensor) and b.numel() == 0) or len(b) == 0:
                continue  

            if torch.is_tensor(img):
                img = F.to_pil_image(img)

            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
            ax.imshow(img)

            # convert to tensor if needed
            if isinstance(b, torch.Tensor):
                b = b.cpu()
            for box in b:
                x1, y1, x2, y2 = box.tolist()
                rect = patches.Rectangle(
                    (x1, y1), x2 - x1, y2 - y1,
                    linewidth=2, edgecolor="red", facecolor="none"
                )
                ax.add_patch(rect)

            ax.set_title(f"Sample {shown}, {len(b)} objects")
            plt.axis("off")
            plt.show()

            shown += 1
            if shown >= n:
                return  # stop once we have shown n samples


In [ ]:
dm.setup("fit")
#show_from_datamodule(dm, n=30, split="train")

In [ ]:
train_loader = dm.train_dataloader()

# Iterate and count
total_elements = 0
for batch in train_loader:
    # 'batch' is a tensor or list of tensors depending on your dataset
    batch_size = len(batch)
    print(f"Processing a batch of size: {batch_size}")
    total_elements += batch_size

print(f"All elements read. Total count: {total_elements}")
